In [1]:
from shiny import App, render, ui, reactive
import pandas as pd
# import faicons

excel_file = pd.ExcelFile("stock.xlsx")

df_inv = pd.read_excel(excel_file, sheet_name="inventory")
df_check_init = pd.read_excel(excel_file, sheet_name="checkouts")
person_dict = pd.read_excel(excel_file, sheet_name="person_dict")
dept_dict = pd.read_excel(excel_file, sheet_name="dept_dict")
df_copies = pd.read_excel(excel_file, sheet_name="copies_dict")

person_list = person_dict["full_name"].unique().tolist()

item_dict = df_inv.set_index("item_id")[["item_name", "price"]].to_dict(orient="index")
inverted_dict = df_inv.set_index("item_name")["item_id"].to_dict()
item_list = df_inv["item_name"].unique().tolist()

add_on_df = df_copies[df_copies["type"] == "add-on"]
add_on_dict = add_on_df.set_index("item_id")["classification"].to_dict()
single_df = df_copies[df_copies["type"] == "single"]
single_dict = single_df.set_index("item_id")["classification"].to_dict()
copies_price = df_copies[["item_id", "type", "price_per_sheet", "classification"]]

nested_dict = {}
for _, row in dept_dict.iterrows():
    dept = row["department"]
    account = row["account"]
    number = row["number"]

    if dept not in nested_dict:
        nested_dict[dept] = {}  # Initialize dictionary for the department

    nested_dict[dept][account] = number

acct_options = [] 
acct_options_copies = []
  
dtype_dict = { "item_name": "object", "quantity": "int64", "cost": "float64","memo": "object", "date": "object", "item_id": "int64", "full_name": "object",}
  